In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 10.4MB/s 
     |████████████████████████████████| 1.1MB 47.8MB/s 
     |████████████████████████████████| 2.9MB 48.5MB/s 
     |████████████████████████████████| 890kB 44.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fc20079e32028f2cc8936a8240df231691c7b88bbfa6c450a88692ace7c2e4ee
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from collections import deque


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
HISTORY_DEPTH = 5
history = deque([], HISTORY_DEPTH)
tokenizer.pad_token = tokenizer.eos_token

# Let's chat for a maximum of 10 lines. Keep the history short.
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    user_input = input(">> User: ")
    if user_input == 'end':
      break

    new_user_input_ids = tokenizer.encode(user_input + ' ' + tokenizer.eos_token, return_tensors='pt')
    print(new_user_input_ids)
    history.appendleft(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat(list(history), dim=-1)

    # generated a response while limiting the total chat history 
    chat_history_ids = model.generate(
        bot_input_ids, 
        max_length=2000, 
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.9,
        temperature = 0.9)
    
    bot_output_ids = chat_history_ids[:, bot_input_ids.shape[-1]:]
    
#    print(f'Chat history shape: {chat_history_ids.shape}')
    print(f'Chat history: {tokenizer.decode(bot_input_ids[:,:][0], skip_special_tokens=False)}')
    
    # pretty print last ouput tokens from bot
    print(f"DialoGPT: {tokenizer.decode(bot_output_ids[0], skip_special_tokens=True)}")
    history.append(bot_output_ids)


>> User: Hey. How you doing?
tensor([[10814,    13,  1374,   345,  1804,    30,   220, 50256]])
Chat history: Hey. How you doing? <|endoftext|>
DialoGPT: I'll get the popcorn.
>> User: Great. Get cokes too. What are we watching?
tensor([[13681,    13,  3497,   269,  3369,  1165,    13,  1867,   389,   356,
          4964,    30,   220, 50256]])
Chat history: Great. Get cokes too. What are we watching? <|endoftext|>Hey. How you doing? <|endoftext|>I'll get the popcorn.<|endoftext|>
DialoGPT: So, are you here?
>> User: Knocking on the door.
tensor([[25095,  8629,   319,   262,  3420,    13,   220, 50256]])
Chat history: Knocking on the door. <|endoftext|>Great. Get cokes too. What are we watching? <|endoftext|>Hey. How you doing? <|endoftext|>I'll get the popcorn.<|endoftext|>So, are you here?<|endoftext|>
DialoGPT: Just let me get my friend here.
>> User: Hurry up. It is starting to rain.
tensor([[   39, 16682,   510,    13,   632,   318,  3599,   284,  6290,    13,
           220, 5025

In [ ]:
print(tokenizer.eos_token_id)

50256


In [ ]:
print(tokenizer.eos_token)

<|endoftext|>
